In [19]:
# Imports

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim

import torch.nn.functional as F

from torchvision import datasets,transforms,models

from collections import OrderedDict

import json

import time

In [20]:
# Load data

data_dir='flowers'
train_dir=data_dir+'/train'
valid_dir=data_dir+'/valid'
test_dir=data_dir+'/test'

train_transforms = transforms.Compose([transforms.RandomRotation(65),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.Resize(224*224),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

valid_transforms=transforms.Compose([transforms.RandomRotation(27),
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.Resize(224*224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_transforms=transforms.Compose([transforms.RandomRotation(94),
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.Resize(224*224),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

train_dataset = datasets.ImageFolder(train_dir,transform=train_transforms)
valid_dataset = datasets.ImageFolder(valid_dir,transform=valid_transforms)
test_dataset= datasets.ImageFolder(test_dir, transform=test_transforms)

trainloader=torch.utils.data.DataLoader(train_dataset,batch_size=8,shuffle=True)
validloader=torch.utils.data.DataLoader(valid_dataset,batch_size=8)
testloader=torch.utils.data.DataLoader(test_dataset,batch_size=8)

In [21]:
# Map labels

with open('cat_to_name.json','r') as f:
    cat_to_name = json.load(f)

In [22]:
# Build network, freeze parameter, recreate classifier

network_model=models.vgg11_bn(pretrained=True)

for p in network_model.parameters():
    p.requires_grad=False

new_classifier=nn.Sequential(OrderedDict([('fc1',nn.Linear(25088,12544)),
                                          ('relu1',nn.ReLU()),
                                          ('fc2',nn.Linear(12544,102)),
                                          ('output',nn.LogSoftmax(dim=1)),
                                          ('dropout',nn.Dropout(p=0.5)),
                                     ]))
network_model.classifier=new_classifier

criterion=nn.NLLLoss()
optimizer=optim.Adam(network_model.classifier.parameters(),lr=0.001)

In [15]:
network_model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): ReLU(inplace)
    (14): MaxPool2d(kernel_size=2, stride=

In [23]:
def training(network_model,trainloader,epochs,print_every,criterion,optimizer,device='cpu'):
    steps=0
    
    network_model.to('cuda')
    for e in range(epochs):
        running_loss=0
        
        for ii,(inputs,labels) in enumerate(trainloader):
            inputs,labels = inputs.to('cuda'),labels.to('cuda')
            
            optimizer.zero_grad()
            
            outputs=network_model.forward(inputs)
            loss=criterion(outputs,labels)
            
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()
            steps+=1
            
            if steps% print_every==0:
                print('Training-----loss: ',running_loss/print_every)
                running_loss=0
                
                network_model.eval()
                with torch.no_grad():
                    valid_loss,valid_accuracy=validation(network_model,validloader,criterion)
                    print('Validation loss: ',valid_loss/len(validloader))
                    print('Validation accuracy: ',valid_accuracy/len(validloader))
                network_model.train()

In [24]:
# Validate trainging
def validation(network_model,validloader,criterion):
    valid_loss=0
    valid_accuracy=0
    
    for image,label in validloader:
        image.resize_(image.shape[0],224*224)
        output=network_model.forward(image)
        valid_loss+=criterion(output,label).item()
        
        valid_accuracy=(label.data==torch.exp(output).max(dim=1)[1]).type(torch.FloatTensor).mean()
        
    return valid_loss,valid_accuracy

In [25]:
# Test trained model
def testing(network_model,testloader):
    test_correct=0
    test_total=0
    
    with torch.no_grad():
        test_image,test_label=iter(testloader).next()
        
        output=network_model(test_image)
        _,predicted=torch.max(output.data,1)
        test_total+=test_label.size(0)
        test_correct+=(predicted==test_label).sum().item()
        
    return print("Test accuracy of network is: ", (test_correct*100)/test_total)

In [26]:
training(network_model,trainloader,3,40,criterion,optimizer,'gpu')
testing(network_model,testloader)

RuntimeError: cuda runtime error (2) : out of memory at c:\programdata\miniconda3\conda-bld\pytorch_1524543037166\work\aten\src\thc\generic/THCStorage.cu:58

In [11]:
torch.cuda.empty_cache()